In [2]:
from Networks import iResNet
from Trainers import split_data,Trainer
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import joblib

In [3]:
# 初始化模型
input_shape = (1,)  # 输入形状
full_connect_shape = (128, 1, 128)  # 全连接层目标形状，根据 true_output0 的形状确定
q = 2  # BB 的数量
N = 3  # 每个 BB 中 SB 的数量

model = iResNet(input_shape, full_connect_shape, q, N, ["T"])

# # 打印模型结构
# print(model)

In [ ]:
# 定义损失函数和优化器
criterions = {"T":nn.MSELoss()}
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# 加载数据
u_train_np = joblib.load('u_train').astype(np.float32)
T_train_np = joblib.load('T_train').astype(np.float32)

u_train = torch.from_numpy(u_train_np).unsqueeze(1)
T_train = torch.from_numpy(T_train_np).unsqueeze(1).unsqueeze(1)

T_train = {"T": T_train}

In [ ]:
# 导入数据
train_loader, val_loader = split_data(u_train, T_train, batch_size=50)
trainer = Trainer(model, optimizer, criterions, device="cuda")

In [ ]:
# 训练
trainer.fit(train_loader, val_loader, num_epochs=50, early_stop_patience=100, save_path="test.pth")

In [ ]:
# 初始化模型
model = iResNet(input_shape, full_connect_shape, q, N, ["T"])
model.load_state_dict(torch.load("test.pth", weights_only=True))
# 将模型移动到设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)